In [1]:
import os 
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq
groq_api = os.getenv("GROQ_API_KEY")

llm = ChatGroq(model="llama-3.3-70b-versatile")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000018AD6ED1940>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000018AD6ED2660>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [10]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain

In [7]:
embedding = OllamaEmbeddings(model="embeddinggemma:300m")

In [33]:
from bs4 import SoupStrainer
loader = WebBaseLoader(web_path=("https://en.wikipedia.org/wiki/World_War_II"))
docs = loader.load()

In [16]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500,chunk_overlap=300)
splits=text_splitter.split_documents(docs)
vector_store=Chroma.from_documents(documents=splits,embedding=embedding)
retriever=vector_store.as_retriever()

In [18]:
system_prompt = (
    "You are an assistant for question answering tasks. "
    "Use the following piece of retrieved context to answer the question. "
    "If you do not know the answer, say that you do not know. "
    "Use three sentences maximum. Keep the answer concise.\n{context}"
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}")
    ]
)

In [19]:
question_answer_chain=create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain(retriever,question_answer_chain)

In [25]:
from IPython.display import Markdown

In [34]:
response=rag_chain.invoke({"input":"how many soldiers were in the red army?"})
display(Markdown(response['answer']))

The exact number of soldiers in the Red Army during World War II is not provided in the given context. However, it is mentioned that the Soviet Union lost around 27 million people, including 8.7 million military deaths.